<a href="https://colab.research.google.com/github/garys-io/crispy-robot/blob/main/Calgary_communities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# !pip install requests
import requests
from pprint import pprint as p
from bs4 import BeautifulSoup
import pathlib
import os

In [18]:
dataDir = '/content/data'

pdfFiles = [file for file in os.listdir(dataDir) if file.endswith('.pdf')]
print(f'Found {len(pdfFiles)} pdf files at dataDir') # ls -al data | grep .pdf | wc -l

# fetch pdf files if there are less than 197 expected
fetchPdfs = len(pdfFiles) < 197
print('fetchPdfs', fetchPdfs)
print()

txtFiles = [file for file in os.listdir(dataDir) if file.endswith('.txt')]
print(f'Found {len(txtFiles)} txt files at dataDir')

# extract text out of pdf files
extractTexts = len(txtFiles) < 197
print('extractTexts', extractTexts)

Found 197 pdf files at dataDir
fetchPdfs False

Found 0 txt files at dataDir
extractTexts True


In [19]:
if fetchPdfs:
  # Gather data
  calgaryCommuniitesUrl = 'https://www.calgary.ca/csps/cns/research-and-strategy/community-profiles/community-profiles.html'

  r = requests.get(calgaryCommuniitesUrl)
  # p(r.text)

  # parse
  soup = BeautifulSoup(r.text, 'html.parser')
  aTags = soup.find_all('a')
  print(f"Found {len(aTags)} anchor tags")

  communityPageUrls = [t.attrs['href'] for t in aTags
                      if 'href' in t.attrs
                      and t.attrs['href'].startswith('https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/')]
  print(f"Found {len(communityPageUrls)} community page URLs")
  # p(aTagsForCommunities)

  # Gather data for individual community by downloading their profile pdf
  for communityPageUrl in communityPageUrls:
    # p(communityPageUrl)
    r = requests.get(communityPageUrl)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfUrls = [t.attrs['href'] for t in soup.find_all('a')
              if t.attrs['href']
              and t.attrs['href'].startswith('/content/')]
    # print(f"Found {len(pdfUrls)} pdf urls")

    for pdfUrl in pdfUrls:
      url = f"https://www.calgary.ca{pdfUrl}"
      filename = pdfUrl.split('/')[-1]
      with open(f'{dataDir}/{filename}', 'wb') as f:
        r = requests.get(url)
        f.write(r.content)
        print(f"Saved: {filename}")
      break # only first pdf is important

In [111]:
# !pip install PyPDF2
import PyPDF2

# TODO : expand to being in extraction of more variables from pdf files as needed
pageParseConfigs = [
  {
    'headerCutoff': 730,
    'footerCutoff': 1185,
    'variableIdxPageInText': {
      'pop': 0,
      'pop 0-14 yr': 5,
      'pop 65+ yr': 7,
      '% spending > 30 % income on shelter': 16,
      '% immigrants': 17,
      'income before tax': 27,
      '% who speak English most often at home': 28
    }
  },
]

parsedFiles = {}

if extractTexts:
  # inspecting first file
  for pdfFile in pdfFiles:
    with open(f'{dataDir}/{pdfFile}', 'rb') as f:
    # with open(f'{dataDir}/abbeydale.pdf', 'rb') as f:
      pdfReader = PyPDF2.PdfFileReader(f)

      parsedPages = []
      for pageIdx, pageParseConfig in enumerate(pageParseConfigs):
        # print(pageParseConfig, pageIdx)
        pageObj = pdfReader.getPage(pageIdx).extractText()
        pageText = pageObj[pageParseConfig['headerCutoff']:pageParseConfig['footerCutoff']].strip()
        pageTextArr = [line for line in pageText.split('\n') if line.strip()]

        page = {}
        for [key, idx] in pageParseConfig['variableIdxPageInText'].items():
        # for [key, idx] in enumerate(range(len(pageTextArr))): # uncomment to gather idx to add to config above
          if len(pageTextArr) > idx:
            page[key] = pageTextArr[idx]
          else:
            pass

        parsedPages.append(page)

      parsedFiles[pdfFile] = parsedPages
    # break

p(parsedFiles)


{'abbeydale.pdf': [{'% immigrants': '33',
                    '% spending > 30 % income on shelter': '21%',
                    '% who speak English most often at home': '72%',
                    'income before tax': '81,232',
                    'pop': '6,150',
                    'pop 0-14 yr': '22',
                    'pop 65+ yr': '8'}],
 'acadia.pdf': [{'% immigrants': '24',
                 '% spending > 30 % income on shelter': '27%',
                 '% who speak English most often at home': '82%',
                 'income before tax': '72,552',
                 'pop': '0,435',
                 'pop 0-14 yr': '13',
                 'pop 65+ yr': '16'}],
 'albert-park-radisson-heights.pdf': [{'% immigrants': '35',
                                       '% spending > 30 % income on shelter': '34%',
                                       'income before tax': '64,429',
                                       'pop': 'n distribution by age in 2016:',
                                